<a href="https://colab.research.google.com/github/iambusra/complingproject/blob/main/Model_Trial_with_Arousal_Values_Only.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
!pip install pandas
!pip install xlsxwriter
!pip install openpyxl

import pandas as pd
import xlsxwriter
from google.colab import files
import io
import numpy as np


In [51]:
# uploading the lexicon txt file which we will use for sentiment analysis via Files section on the left


In [52]:
# turning the file into a pandas data frame called lexicon

lexicon = pd.read_csv("Turkish-tr-NRC-VAD-Lexicon.txt", sep="\t", error_bad_lines=False)

In [53]:
# uploading news data via Files section on the left

In [54]:
# turning the file into a data frame called news
news = pd.read_excel("data_clean.xlsx")

# removing the first two columns, which do not contain any relevant information
news = news.drop(['Unnamed: 0', 'Haber Başlıkları'], axis = 1)
print(news)

                                               Linkler                                           Metinler
0                                    [Google Haberler]                                                NaN
1    http://www.haberinadresi.com/bursa-da-kadin-su...  [Bursa’da kimliği belirsiz bir kişi, gözüne ke...
2    http://www.pusulagazetesi.com.tr/kadin-surucu-...  Kadın sürücü kaza yaptı. Zonguldak’ta Kapuz Ma...
3    http://www.pusulagazetesi.com.tr/kadin-surucu-...  Kadın sürücü ve annesi yaralandı, kaza anı kam...
4    http://www.pusulagazetesi.com.tr/kaza-yapan-ka...  Zonguldak’ın Ereğli ilçesinde sürücünün direks...
..                                                 ...                                                ...
226  https://www.sozcu.com.tr/2021/gundem/alkollu-k...  İzmir'in Konak ilçesinde, otomobili uygulama n...
227  https://www.sozcu.com.tr/2021/gundem/seyahat-i...  Edinilen bilgilere göre, ilçe dışından gelen v...
228  https://www.sozcu.com.tr/2021/gunun-icind

In [55]:
news.describe()

,Linkler,Metinler
count,231,227
unique,222,219
top,https://www.milliyet.com.tr/milliyet-tv/kadinl...,[]
freq,2,4


In [56]:
# upload also sürücü data

# turning the file into a data frame called news
news_sürücü = pd.read_excel("/content/data_surucu.xlsx")

# removing the first two columns, which do not contain any relevant information
news_sürücü = news_sürücü.drop(['Unnamed: 0', 'Haber Başlıkları'], axis = 1)
print(news_sürücü)

                                              Linkler                                           Metinler
0                                   [Google Haberler]                                                NaN
1   http://www.adayorum.com/alkolltrafige-kapali-y...  Sakarya’nın Sapanca ilçesinde alkollü sürücü i...
2   http://www.boluolay.com/bolu/dur-ihtarina-uyma...  Polisin dur ihtarına uymayarak kaçmaya başlaya...
3   https://www.sakaryadanhaber.com/haber/7262954/...  Kocaeli’nin Gebze ilçesinde 8 yaşındaki çocuk,...
4   https://www.golcukgundem.com/haber/7259308/alk...  Alkollü olduğu iddia edilen sürücü aşırı hız y...
..                                                ...                                                ...
93  https://tr.sputniknews.com/abd/202105271044596...  ABD’de geçtiğimiz pazar günü önce telefonda po...
94  https://shiftdelete.net/nvidia-bazi-ekran-kart...  SDN Network, SDN Teknoloji Haberleri, Mediatre...
95  https://www.haberturk.com/kayseri-haberleri/87...  

In [57]:
news_sürücü["label"] = "neutral"
news["label"] = "biased"
news = news.append(news_sürücü)

In [58]:
news.describe()

,Linkler,Metinler,label
count,329,321,329
unique,318,313,2
top,https://www.hurriyet.com.tr/gundem/kadin-suruc...,[],biased
freq,2,4,231


In [59]:
# there are duplicates in Linkler and Metinler column. Let's remove duplicates from the data
news = news.drop_duplicates(subset='Linkler')
news = news.drop_duplicates(subset='Metinler')
news.describe()

,Linkler,Metinler,label
count,306,305,306
unique,306,305,2
top,https://www.dha.com.tr/yurt/esi-endiselendi-ka...,"Kadın sürücü, çarptığı otomobile takla attırdı...",biased
freq,1,1,213


Yes! We do not have any duplicates anymore. Now we can go on to code our labels as integers for our model. Let's stick to alphabetical order and code "biased" as 0 and "neutral" as 1.

In [60]:
labels = {'biased':0,'neutral':1}
news['label'] = news.label.map(labels)

news[205:220]

,Linkler,Metinler,label
222,https://www.sabah.com.tr/yasam/2021/06/02/duzc...,"Düzce'de, TEM Otoyolu’nda hafif ticari araçla ...",0
223,https://www.sabah.com.tr/yasam/2021/06/03/kirm...,Antalya'da kırmızı ışık ihlali yapan kadın sür...,0
224,https://www.sondakika.com/haber/haber-kisitlam...,Basın Ekspres yolunda kısıtlama gününde başka ...,0
225,https://www.sondakika.com/haber/haber-son-daki...,Son dakika habere göre Bisikletli çocuğa çarpı...,0
226,https://www.sozcu.com.tr/2021/gundem/alkollu-k...,"İzmir'in Konak ilçesinde, otomobili uygulama n...",0
227,https://www.sozcu.com.tr/2021/gundem/seyahat-i...,"Edinilen bilgilere göre, ilçe dışından gelen v...",0
229,https://www.trthaber.com/haber/guncel/kentin-t...,Kentin tek kadın ambulans şoförü oldu\nKayseri...,0
230,https://www.yenisafak.com/hayat/akasya-duragin...,"Osmaniye'de Döndü Bahargülü, takip ettiği dizi...",0
1,http://www.adayorum.com/alkolltrafige-kapali-y...,Sakarya’nın Sapanca ilçesinde alkollü sürücü i...,1
2,http://www.boluolay.com/bolu/dur-ihtarina-uyma...,Polisin dur ihtarına uymayarak kaçmaya başlaya...,1


In [61]:
# Let's drop NaN rows and shuffle the rows so that neutral and biased rows are mixed
news = news.dropna()
news = news.sample(frac = 1)
news.head(10)

,Linkler,Metinler,label
213,https://www.haberturk.com/antalya-haberleri/87...,Antalya’da virajı alamayarak takla atan ve ref...,0
52,https://www.gursesgazetesi.com/yasam/manisada-...,", 1 kişinin feci şekilde can verdiği trafik ka...",1
128,https://www.milliyet.com.tr/milliyet-tv/aracta...,Araçtan yola atılan yavru kediyi kadın sürücü ...,0
33,https://www.ensonhaber.com/3-sayfa/adanada-ehl...,[Adana'nın Seyhan ilçesine bağlı Uçak Mahalles...,0
27,https://www.denizli24haber.com/haber/fethiyede...,"[Denizli İl Kültür ve Turizm Müdürünün ba..., ...",0
18,http://www.marmaragazetesi.com/dur-ihtarina-uy...,Samsun'un Bafra ilçesinde 'dur' ihtarına uymay...,1
46,https://www.hurriyet.com.tr/gundem/beylikduzun...,"[HÜRRİYET'E BAĞLI KAL, E-Gazete ile tüm gelişm...",0
170,https://www.haberler.com/korna-calan-kadin-sur...,Korna çalan kadın sürücüyü darp eden şüpheli y...,0
39,https://www.haberdenizli.com/m/guncel/denizli-...,[\n\n\n\r\n(adsbygoogle = window.adsbygoogle |...,0
90,https://www.milliyet.com.tr/gundem/yolda-durdu...,Yolda durduruldu! Şüphe gerçek çıktı...\nBursa...,0


In [62]:
# preprocessing function that we have written in the class.
# We are using Turkish stop words this time since our data is in Turkish.

import nltk
import string
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords

stopwords = set(stopwords.words('turkish'))

def PreprocessNews(text):
  text = text.lower()
  text = text.translate(str.maketrans('', '', string.punctuation))
  tokenized_text = nltk.word_tokenize(text)
  clean_text = [word for word in tokenized_text if word not in stopwords ]
  return(clean_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [63]:
# to make sure that every row in Metinler is of type str for preprocessing
# news.Metinler = news.Metinler.astype(str)

In [64]:
# preprocessing our news bodies
news['Metinler'] = news.apply(lambda x: PreprocessNews(x['Metinler']),axis=1)

In [65]:
news.head(10)

,Linkler,Metinler,label
213,https://www.haberturk.com/antalya-haberleri/87...,"[antalya, ’, virajı, alamayarak, takla, atan, ...",0
52,https://www.gursesgazetesi.com/yasam/manisada-...,"[1, kişinin, feci, şekilde, can, verdiği, traf...",1
128,https://www.milliyet.com.tr/milliyet-tv/aracta...,"[araçtan, yola, atılan, yavru, kediyi, kadın, ...",0
33,https://www.ensonhaber.com/3-sayfa/adanada-ehl...,"[adananın, seyhan, ilçesine, bağlı, uçak, maha...",0
27,https://www.denizli24haber.com/haber/fethiyede...,"[denizli, i̇l, kültür, turizm, müdürünün, ba, ...",0
18,http://www.marmaragazetesi.com/dur-ihtarina-uy...,"[samsunun, bafra, ilçesinde, dur, ihtarına, uy...",1
46,https://www.hurriyet.com.tr/gundem/beylikduzun...,"[hürri̇yete, bağli, kal, egazete, gelişmeler, ...",0
170,https://www.haberler.com/korna-calan-kadin-sur...,"[korna, çalan, kadın, sürücüyü, darp, eden, şü...",0
39,https://www.haberdenizli.com/m/guncel/denizli-...,"[adsbygoogle, windowadsbygoogle, push, haberde...",0
90,https://www.milliyet.com.tr/gundem/yolda-durdu...,"[yolda, durduruldu, şüphe, gerçek, çıktı, burs...",0


# Let's find the words that are present both in the Lexicon and the news bodies.

In [66]:
# a loop that will save to matching_words list the tokens in the news bodies which have a matching entry in the lexicon.
matching_words = []
for sentence in metinler:
  for word in sentence:
    if word in lexicon["Turkish-tr"].tolist():
      matching_words.append(word)

# It took 15 seconds to complete. Let's check the first 50 matching tokens.
matching_words[0:50]

NameError: ignored

**How many words in our news bodies will end up having sentiment values by virtue of being listed in the lexicon?**

In [ ]:
len(matching_words)

**We have 11477 tokens in news bodies which are listed in the lexicon. Cool!**

**And how many types?**

In [ ]:
len(np.unique(matching_words))

**We have 1206 types in news bodies that are in the Sentiment lexicon and this is the number we get without using any morphological analyzer. We will incorporate an analyzer (as Büşra Marşan's or a better one) to increase this number.**

This notebook was sort of an exploratory analysis to see if our lexicon will be useful or not. Based on the number of matching lexical items, this lexicon seems promising at least for this project. Now, what we further need to do is to find the matching lexical items in our lexicon for all words (including the morphologically complex ones) in our data and assign them the corresponding sentiment values.

Then, we will move on to randomly separate our data so that we have different data sets for both training and testing phases. Also, we will categorize the news (supervised learning) as biased or neutral in order to teach our model how to categorize them by analyzing the features (sentiment values) in the data frame.

Finally, we will give the classifier we built the uncategorized test data and see if it can classify them based on sentiment values. We will evaluate the classifier on different dimensions like accuracy and precision by using scikitlearn.

# Assigning Sentiment Values to Tokens in the News Body

In [67]:
# create empty lists for saving sentiment values in the loop below
A = []
D = []
V =[]

# create variables with type int and value 0 so that we can add the sentiment values
# for each token to them and return the total sentiment values of a news body in the end.


# Look at the news body one by one (one row at a time).
# Then for the news body in that row, find sentiment value for each word.
# Add them up until moving onto the next news body in the next row.
for metin in news['Metinler']:
  for word in metin:
    if word in lexicon["Turkish-tr"].tolist():
      arousal = 0
      dominance = 0
      valence = 0
      arousal += lexicon["Arousal"][lexicon["Turkish-tr"].tolist().index(word)]
      dominance += lexicon["Dominance"][lexicon["Turkish-tr"].tolist().index(word)]
      valence += lexicon["Valence"][lexicon["Turkish-tr"].tolist().index(word)]

  A.append([arousal])
  D.append([dominance])
  V.append([valence])



print(A)
print(D)
print(V)

[[0.292], [0.324], [0.434], [0.385], [0.33], [0.235], [0.33399999999999996], [0.461], [0.292], [0.314], [0.77], [0.51], [0.292], [0.42], [0.33399999999999996], [0.32799999999999996], [0.287], [0.447], [0.33], [0.51], [0.511], [0.235], [0.37], [0.25], [0.5489999999999999], [0.361], [0.5589999999999999], [0.594], [0.295], [0.461], [0.368], [0.49200000000000005], [0.99], [0.48], [0.36], [0.6890000000000001], [0.25], [0.716], [0.47200000000000003], [0.436], [0.33299999999999996], [0.62], [0.368], [0.25], [0.33], [0.292], [0.44299999999999995], [0.42], [0.25], [0.433], [0.37], [0.33], [0.51], [0.5770000000000001], [0.51], [0.5589999999999999], [0.565], [0.235], [0.318], [0.35], [0.565], [0.34], [0.2], [0.392], [0.33], [0.68], [0.625], [0.318], [0.28800000000000003], [0.33399999999999996], [0.33399999999999996], [0.24100000000000002], [0.368], [0.42], [0.363], [0.375], [0.235], [0.33299999999999996], [0.37], [0.24], [0.43700000000000006], [0.409], [0.17300000000000001], [0.276], [0.332999999

**NOT SURE IF WE NEED TO TURN SENTIMENT VALUES INTO FLOATS RATHER THAN LEAVING THEM AS LISTS**

In [ ]:
# Check if the number of rows in the news match the number of items in the lists A, D, and V.
print(len(A))
print(len(D))
print(len(V))

# 305 rows as expected. Great!

In [68]:
# Let's append these lists with sentiment values to news data frame.
news["Arousal"] = A
news["Dominance"] = D
news["Valence"] = V
news.head()

,Linkler,Metinler,label,Arousal,Dominance,Valence
213,https://www.haberturk.com/antalya-haberleri/87...,"[antalya, ’, virajı, alamayarak, takla, atan, ...",0,[0.292],[0.37],[0.542]
52,https://www.gursesgazetesi.com/yasam/manisada-...,"[1, kişinin, feci, şekilde, can, verdiği, traf...",1,[0.324],[0.565],[0.552]
128,https://www.milliyet.com.tr/milliyet-tv/aracta...,"[araçtan, yola, atılan, yavru, kediyi, kadın, ...",0,[0.434],[0.779],[0.91]
33,https://www.ensonhaber.com/3-sayfa/adanada-ehl...,"[adananın, seyhan, ilçesine, bağlı, uçak, maha...",0,[0.385],[0.43799999999999994],[0.49]
27,https://www.denizli24haber.com/haber/fethiyede...,"[denizli, i̇l, kültür, turizm, müdürünün, ba, ...",0,[0.33],[0.46399999999999997],[0.5]


In [69]:
# saving the data frame with sentiment values
writer = pd.ExcelWriter('Sentiment_data_clean_surucu.xlsx')
news.to_excel(writer)
writer.save()

# Arousal Modeling Trial



There are 305 rows. 20% of the data should go to test and the rest to training.

*   x_train = training input data
*   y_train = training labels
*   x_test = test input data
*   y_test = test labels 

In [ ]:
test_rows = news.sample(frac = 0.2, random_state = 42)

print(test_rows)

61 rows. Let's get the remaining as the training data.

In [ ]:
training_rows = news.drop(test_rows.index)

print(training_rows)

Training data has 217 rows. It makes 278 in total. Where is the rest? 27 rows vanished?!

In [95]:
# subsetting the relevant input output for test data

x_test = test_rows["Arousal"]
y_test = test_rows["label"]

print(x_test)
print(y_test)

99                   [0.25]
227                 [0.428]
42                   [0.33]
136                 [0.519]
189                 [0.565]
               ...         
81                  [0.529]
18                   [0.33]
71                  [0.565]
43                    [0.5]
131    [0.5660000000000001]
Name: Arousal, Length: 61, dtype: object
99     0
227    0
42     1
136    0
189    0
      ..
81     1
18     0
71     1
43     1
131    0
Name: label, Length: 61, dtype: int64


In [ ]:
# subsetting the relevant input(x) output(y) for training data

x_train = training_rows["Arousal"]
y_train = training_rows["label"]

print(x_training)
print(y_training)

In [80]:
# import CountVectorizer and create an instance of the object 
from sklearn.feature_extraction.text import CountVectorizer
bag_of_words = CountVectorizer(analyzer = lambda x: x)
training_data = bag_of_words.fit_transform(x_train)

In [ ]:
# let us print the size of our vocabulary, namely Arousal Values
print('vocabulary size: {}'.format(len(bag_of_words.get_feature_names())))

# let us print 20 words from the vocabulary
bag_of_words.get_feature_names()[50:70]

In [86]:
# let's see the test data input
test_data = bag_of_words.transform(x_test)

In [87]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB()
naive_bayes.fit(training_data, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [88]:
predictions = naive_bayes.predict(test_data)

In [89]:
predictions[0:50]

array([1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0])

In [ ]:
list(x_test)[0:50]

In [ ]:
print(test_data)

In [ ]:
test_rows["Predictions"] = predictions

In [98]:
writeit = pd.ExcelWriter('test_predictions.xlsx')
test_rows.to_excel(writeit)
writeit.save()

Sorry for the mess but the codes in the last part are just to see the testing data frame in an excel to compare the predictions and the original labels along with the nes bodies. 43 correct predictions out of 61 labels. Decent. Next we should try to work out a model consisting all 3 sentiment values as the input and the label as the output.